In [31]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
import random
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
to_generate_data = False
to_train = True
to_test = True

*Data Preparation*

In [6]:
if to_generate_data:
    df = pd.read_parquet('autodl-tmp/mins_data.parquet')
    df.sort_values(by=['START_TIME', 'Attraction'], inplace=True)
    df['lag1'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(1).fillna(0)
    df['lag2'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(2).fillna(0)
    df.reset_index(inplace = True,drop = True)
    one_hot_columns = [col for col in df.columns if 
                       pd.api.types.is_numeric_dtype(df[col]) and 
                       sorted(df[col].unique()) == [0, 1]]
    non_one_hot_columns = [col for col in df.columns if 
                           pd.api.types.is_numeric_dtype(df[col]) and 
                           col not in one_hot_columns and col not in ['lag1','lag2','WAIT_TIME_MAX']]
    scaler = StandardScaler()
    df[non_one_hot_columns] = scaler.fit_transform(df[non_one_hot_columns])
    
    num_attractions = len(df.Attraction.unique())
    grouped = df.groupby('START_TIME')
    feature_columns = [i for i in list(df) if i not in ['WORK_DATE',
     'START_TIME',
     'Attraction',
     'WAIT_TIME_MAX',
     'DEB_TIME',
     'FIN_TIME',
     'DEB_TIME_x',
     'FIN_TIME_x',
     'DEB_TIME_y',
     'FIN_TIME_y']]
    zero_data = np.zeros((1, len(feature_columns)))
    zero_df = pd.DataFrame(zero_data, columns=feature_columns)
    processed_groups = []
    targets = []
    all_attractions = sorted(df['Attraction'].unique())
    for time_point, group in tqdm(grouped):
        processed_group = group.copy()
        processed_group = processed_group.drop_duplicates(subset=['Attraction'], keep='first')
        present_attractions = group['Attraction'].unique()
        missing_attractions = set(all_attractions) - set(present_attractions)
        for attraction in missing_attractions:
            temp_df = zero_df.copy()
            temp_df['Attraction'] = attraction
            temp_df['START_TIME'] = time_point
            processed_group = pd.concat([processed_group, temp_df], ignore_index=True)
        processed_group.sort_values(by=['Attraction'], inplace=True)
        t = processed_group.pop('WAIT_TIME_MAX').fillna(0)
        data = torch.tensor(processed_group[feature_columns].values, dtype=torch.float)
        processed_groups.append(data)
        targets.append(t)
        if data.shape[0] != 26:
            break
    features_tensor = torch.stack(processed_groups)
    labels_tensor = torch.tensor(np.array(targets))
    total_samples = len(features_tensor)
    # last 10% for validation
    split_idx = int(total_samples * 0.9)
    
    train_features = features_tensor[:split_idx]
    train_labels = labels_tensor[:split_idx]
    validation_features = features_tensor[split_idx:]
    validation_labels = labels_tensor[split_idx:]
    
    train_dataset = TensorDataset(train_features, train_labels)
    validation_dataset = TensorDataset(validation_features, validation_labels)
    torch.save(train_dataset, 'autodl-tmp/train_dataset.pth')
    torch.save(validation_dataset, 'autodl-tmp/validation_dataset.pth')
else:
    df = pd.read_parquet('autodl-tmp/mins_data.parquet')
    df.sort_values(by=['START_TIME', 'Attraction'], inplace=True)
    df['lag1'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(1).fillna(0)
    df['lag2'] = df.groupby('Attraction')['WAIT_TIME_MAX'].shift(2).fillna(0)
    df.reset_index(inplace = True,drop = True)
    one_hot_columns = [col for col in df.columns if 
                       pd.api.types.is_numeric_dtype(df[col]) and 
                       sorted(df[col].unique()) == [0, 1]]
    non_one_hot_columns = [col for col in df.columns if 
                           pd.api.types.is_numeric_dtype(df[col]) and 
                           col not in one_hot_columns and col not in ['lag1','lag2','WAIT_TIME_MAX']]
    scaler = StandardScaler()
    df[non_one_hot_columns] = scaler.fit_transform(df[non_one_hot_columns])
    
    num_attractions = len(df.Attraction.unique())
    grouped = df.groupby('START_TIME')
    feature_columns = [i for i in list(df) if i not in ['WORK_DATE',
     'START_TIME',
     'Attraction',
     'WAIT_TIME_MAX',
     'DEB_TIME',
     'FIN_TIME',
     'DEB_TIME_x',
     'FIN_TIME_x',
     'DEB_TIME_y',
     'FIN_TIME_y']]
    train_dataset = torch.load('autodl-tmp/train_dataset.pth')
    validation_dataset = torch.load('autodl-tmp/validation_dataset.pth')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False)

*Network design*

In [7]:
class Network1(nn.Module):
    #version 1: input-1500-1000-output
    #version 2: input-1500-1500-1000-output
    def __init__(self, input_size, output_size, num_layers=1):
        super(Network1, self).__init__()
        self.fc1 = nn.Linear(input_size, 1500)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1500, 1500)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(1500, 1000)
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(1000, output_size)
        self.relu = nn.LeakyReLU()
        init.kaiming_normal_(self.fc1.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc4.weight, nonlinearity='leaky_relu')
    def forward(self, x, return_intermediate=False):
        intermediate_outputs = {}
        out = self.relu(self.fc1(x))
        intermediate_outputs['fc1'] = out
        out = self.dropout1(out)
        intermediate_outputs['dropout1'] = out
        out = self.relu(self.fc2(out))
        intermediate_outputs['fc2'] = out
        out = self.dropout2(out)
        intermediate_outputs['dropout2'] = out
        out = self.relu(self.fc3(out))
        intermediate_outputs['fc3'] = out
        out = self.dropout3(out)
        intermediate_outputs['dropout3'] = out
        out = self.fc4(out)
        intermediate_outputs['fc4'] = out

        if return_intermediate:
            return intermediate_outputs
        else:
            return out

class Network2(nn.Module):
    def __init__(self,input_size,output_size):
        super(Network2, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 256)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256,128)
        self.dropout3 = nn.Dropout(0.3)
        self.fc4 = nn.Linear(128, output_size)
        self.relu = nn.LeakyReLU()
        init.kaiming_normal_(self.fc1.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='leaky_relu')
        init.kaiming_normal_(self.fc4.weight, nonlinearity='leaky_relu')
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout1(out)
        out = self.relu(self.fc2(out))
        out = self.dropout2(out)
        out = self.relu(self.fc3(out))
        out = self.dropout3(out)
        out = self.fc4(out)
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network1 = Network1(input_size=num_attractions*len(feature_columns),
                    output_size=num_attractions*num_attractions).to(device)
network2 = Network2(input_size=num_attractions, output_size=num_attractions).to(device)


In [33]:
def build_initial_matrix(out):
    matrix = out.view(26, 26)
    normalized_matrix = matrix / matrix.sum(dim=1, keepdim=True)
    return normalized_matrix
def calculate_stationary_matrix(matrix, num_iterations=30, epsilon=1e-6):
    v = torch.ones(matrix.size(0), dtype=matrix.dtype, device=matrix.device)
    v = v / v.sum()
    for _ in range(num_iterations):
        v_next = torch.mv(matrix, v)
        v_next = v_next / v_next.sum()
        if torch.norm(v - v_next) < epsilon:
            break
        v = v_next
    return v

def Middle(out):
    return calculate_stationary_matrix(build_initial_matrix(out))

In [9]:
if to_train:
    criterion = nn.MSELoss()
    optimizer1 = optim.Adam(network1.parameters(), lr=0.001)
    optimizer2 = optim.Adam(network2.parameters(), lr=0.001)

*Evaluation Design*

In [34]:
def validate(model1, model2, validation_loader, criterion, device):
    model1.eval() 
    model2.eval()
    val_loss = 0.0
    with torch.no_grad(): 
        for inputs, targets in validation_loader:
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            out1 = model1(inputs)
            intermediate_matrix = build_initial_matrix(out1)
            stationary_matrix = calculate_stationary_matrix(intermediate_matrix)
            predictions = model2(stationary_matrix)
            loss = criterion(predictions, targets)
            val_loss += loss.item()
    return np.sqrt(val_loss / len(validation_loader))
def RMSE_for_Attraction(network1,network2):
    attraction = df.Attraction.unique()
    attraction.sort()
    network1.eval() 
    network2.eval()
    result = pd.DataFrame({'attraction':attraction,'RMSE':[0 for i in attraction]})
    val_loss = 0.0
    with torch.no_grad(): 
        for inputs, targets in tqdm(validation_loader):
            targets = targets.reshape(26)
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            out1 = network1(inputs)
            intermediate_matrix = build_initial_matrix(out1)
            stationary_matrix = calculate_stationary_matrix(intermediate_matrix)
            predictions = network2(stationary_matrix)
            for i in range(len(predictions)):
                index = result['attraction'] == attraction[i]
                result.loc[index, 'RMSE'] += float((targets[i] - predictions[i]))**2
    result.RMSE = result.RMSE/len(validation_loader)
    result.RMSE = result.RMSE.apply(lambda x:np.sqrt(x))
    return result

*Model Training*

In [11]:
if to_train:
    num_epochs = 10
    current_best = 1e10
    RMSE_val = []
    for epoch in range(num_epochs):
        network1.train()
        network2.train()
        train_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")
        for batch_idx, (inputs, targets) in pbar:
            inputs = inputs.reshape((1,num_attractions*len(feature_columns))).float()
            inputs, targets = inputs.to(device), targets.float().to(device)
            optimizer1.zero_grad()
            optimizer2.zero_grad()
            out1 = network1(inputs)
            stationary_matrix = Middle(out1)
            predictions = network2(stationary_matrix).reshape((1,26)).float()
            loss = criterion(predictions, targets)
            loss.backward()
            clip_value = 1 #1-10
            nn.utils.clip_grad_norm_(network1.parameters(), clip_value) 
            nn.utils.clip_grad_norm_(network2.parameters(), clip_value)
            optimizer1.step()
            optimizer2.step()
            train_loss += loss.item()
            pbar.set_postfix({'Train Loss': train_loss / (batch_idx + 1)})
            
        val_loss = validate(network1, network2, validation_loader, criterion, device)
        RMSE_val.append(val_loss)
        if np.sqrt(val_loss) < current_best:
            torch.save(network1, 'network1_v2.pth')
            torch.save(network2, 'network2_v2.pth')
            current_best = val_loss
        print(f"Validation Loss: {val_loss:.4f}")

Epoch 1: 100%|██████████| 36540/36540 [05:24<00:00, 112.78it/s, Train Loss=421] 


Validation Loss: 20.9778


Epoch 2: 100%|██████████| 36540/36540 [06:03<00:00, 100.58it/s, Train Loss=455]


Validation Loss: 20.8362


Epoch 3: 100%|██████████| 36540/36540 [05:38<00:00, 107.89it/s, Train Loss=418]


...

Validation Loss: 18.2693


Epoch 9: 100%|██████████| 36540/36540 [07:43<00:00, 78.81it/s, Train Loss=4.5e+6]  


Validation Loss: 26.5229


Epoch 10: 100%|██████████| 36540/36540 [07:58<00:00, 76.40it/s, Train Loss=4.48e+6] 


Validation Loss: 28.2354


*Best Model Evaluation*

In [40]:
if to_test:
    # best_network1 = torch.load('network1_v2.pth')
    # best_network2 = torch.load('network2_v2.pth')
    # result = RMSE_for_Attraction(best_network1,best_network2)
    print('RMSE for all Attractions', result.RMSE.mean())
    print(result)

RMSE for all Attractions 17.653672616149745
          attraction       RMSE
0        Bumper Cars   7.623577
1        Bungee Jump  20.003759
2       Circus Train   3.888419
3        Crazy Dance   3.705589
4      Dizzy Dropper  10.570900
5         Drop Tower  21.876137
6     Flying Coaster  14.044068
7          Free Fall  44.365342
8        Giant Wheel  21.154676
9       Giga Coaster   9.930772
10          Go-Karts  21.436050
11     Haunted House  14.451574
12     Himalaya Ride   2.787177
13  Inverted Coaster  22.863536
14    Kiddie Coaster  11.530456
15    Merry Go Round  13.787891
16        Oz Theatre   5.651201
17       Rapids Ride  17.194991
18    Roller Coaster  23.407296
19  Spinning Coaster  14.330277
20      Spiral Slide  79.192839
21     Superman Ride   7.905478
22        Swing Ride  21.931852
23     Vertical Drop   8.854938
24        Water Ride  15.296954
25           Zipline  21.209739


In [42]:
result.to_csv('version2result.csv',index = False)

*Result Combination*

In [60]:
result1 = pd.read_csv('version1result.csv')
result2 = pd.read_csv('version2result.csv')
result = pd.merge(result1,result2,on = 'attraction')
result = result.drop(columns = ['Unnamed: 0_x','Unnamed: 0_y'])
result = result.rename(columns = {'RMSE_x':'Markov1','RMSE_y':'Markov2'})
result['Combined_RMSE'] = result[['Markov1', 'Markov2']].min(axis=1)
result['Chosed_Model'] = np.where(result['Combined_RMSE'] == result['Markov1'], 'Markov1', 'Markov2')

In [62]:
print('Combined Model RMSE:',result.Combined_RMSE.mean())
result

Combined Model RMSE: 17.026918440768203


,attraction,Markov1,Markov2,Combined_RMSE,Chosed_Model
0,Bumper Cars,12.496342,7.623577,7.623577,Markov2
1,Bungee Jump,28.619547,20.003759,20.003759,Markov2
2,Circus Train,6.192073,3.888419,3.888419,Markov2
3,Crazy Dance,0.044842,3.705589,0.044842,Markov1
4,Dizzy Dropper,16.778949,10.570900,10.570900,Markov2
5,Drop Tower,35.348845,21.876137,21.876137,Markov2
6,Flying Coaster,20.745494,14.044068,14.044068,Markov2
7,Free Fall,49.920049,44.365342,44.365342,Markov2
8,Giant Wheel,49.385965,21.154676,21.154676,Markov2
9,Giga Coaster,0.070613,9.930772,0.070613,Markov1


In [46]:
result2.to_csv('version2result.csv',index = False)

In [67]:
result[~result['attraction'].isin(['Spiral Slide', 'Free Fall'])].Combined_RMSE.mean()

13.297570765424718